# Preprocessing

After having done $2$ assignments, I have deemed it fit to factor out preprocessing into its own notebook, instead of having it attached to the first model so as to not hinder its execution or add overhead.

The data required to be used by the models shall be stored under `<arbitrary path>`.

## Data Loading

Loading the `.parquet` files for training. I am unsure which language to try first..

In [1]:
import os
import polars as pl

In [2]:
# Select which language
# we want to train for
lang = 'italian'

# Prepare the path
training_data_path = os.path.join('..', 'data', 'input_lang', lang, 'training.parquet')

In [3]:
raw_parquet_data = pl.read_parquet(training_data_path)

raw_parquet_data.head()

text
str
"""- Grazie, amico. ###>- Thanks,…"
"""Dillo. ###>Say it."""
"""Trifosfato di sodio (tripolifo…"
"""Invero è avido per amore delle…"
"""ALLEGATO I ###>ANNEX I"""


In [4]:
split_data = raw_parquet_data.select(
    raw_parquet_data["text"]
    .str.split(" ###>")
    .list.to_struct(n_field_strategy = "max_width",
                    fields = [lang, "english"])
    .alias("splitten")
).unnest("splitten")

split_data.head()

italian,english
str,str
"""- Grazie, amico.""","""- Thanks, buddy."""
"""Dillo.""","""Say it."""
"""Trifosfato di sodio (tripolifo…","""Sodium triphosphate (sodium tr…"
"""Invero è avido per amore delle…","""Surely, he is ardent in his lo…"
"""ALLEGATO I""","""ANNEX I"""
